# Parameter Estimation in CONFLUENCE

## Introduction

This notebook focuses on the crucial task of parameter estimation for hydrological models in CONFLUENCE. Accurate parameter estimation is essential for model performance and reliability in simulating hydrological processes.

Key aspects covered in this notebook include:

1. Setting up calibration experiments
2. Running calibration algorithms
3. Evaluating calibration results

In this notebook we explore various techniques for parameter estimation. By the end of this notebook, you will have a calibrated model ready for scenario analysis, forecasting, or other applications in your hydrological study.

## First we import the libraries and functions we need

In [1]:
import sys
from pathlib import Path
from typing import Dict, Any
import logging
import yaml # type: ignore
import subprocess

current_dir = Path.cwd()
parent_dir = current_dir.parent.parent
sys.path.append(str(parent_dir))

#from utils.optimization_utils.ostrich_util import OstrichOptimizer

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

## Check configurations

Now we should print our configuration settings and make sure that we have defined all the settings we need. 

In [2]:
config_path = Path('../../0_config_files/config_active.yaml')
with open(config_path, 'r') as config_file:
    config = yaml.safe_load(config_file)
    print(f"FORCING_DATASET: {config['FORCING_DATASET']}")
    print(f"EASYMORE_CLIENT: {config['EASYMORE_CLIENT']}")
    print(f"FORCING_VARIABLES: {config['FORCING_VARIABLES']}")
    print(f"EXPERIMENT_TIME_START: {config['EXPERIMENT_TIME_START']}")
    print(f"EXPERIMENT_TIME_START: {config['EXPERIMENT_TIME_START']}")

FORCING_DATASET: ERA5
EASYMORE_CLIENT: easymore cli
FORCING_VARIABLES: longitude,latitude,time,LWRadAtm,SWRadAtm,pptrate,airpres,airtemp,spechum,windspd
EXPERIMENT_TIME_START: 2010-01-01 01:00
EXPERIMENT_TIME_START: 2010-01-01 01:00


## Define default paths

Now let's define the paths to data directories before we run the pre processing scripts and create the containing directories

In [3]:
# Main project directory
data_dir = config['CONFLUENCE_DATA_DIR']
project_dir = Path(data_dir) / f"domain_{config['DOMAIN_NAME']}"

# Data directoris
simulation_dir = project_dir / 'simulations' / f"{config['EXPERIMENT_ID']}"
hydro_model_dir = simulation_dir / f"{config['HYDROLOGICAL_MODEL']}"
routing_model_dir = simulation_dir / f"{config['ROUTING_MODEL']}"

# Make sure the new directories exists
hydro_model_dir.mkdir(parents = True, exist_ok = True)
routing_model_dir.mkdir(parents = True, exist_ok = True)

## 1. Run parameter estimation 

In [ ]:
# Calibrate the model using specified method and objectives
if config.get('OPTMIZATION_ALOGORITHM') == 'OSTRICH':
    optimizer = OstrichOptimizer(config, logger)
    optimizer.run_optimization()

else:
    config_path = Path(config.get('CONFLUENCE_CODE_DIR')) / '0_config_files' / 'config_active.yaml'

    cmd = [
        'srun',
        '-n', str(config.get('MPI_PROCESSES')),
        'python',
        str(Path(f"{config['CONFLUENCE_CODE_DIR']}") / 'utils' / 'optimization_utils' / 'parallel_parameter_estimation.py'), 
        str(config_path)
    ]

    try:
        subprocess.run(cmd, check=True)
    except subprocess.CalledProcessError as e:
        logger.error(f"Error running parallel optimization: {e}")

INFO: parallel_parameter_estimation.main(): Process 0 initialized
INFO: parallel_parameter_estimation.main(): Process 1 initialized
INFO: parallel_parameter_estimation.main(): Process 2 initialized
INFO: parallel_parameter_estimation.main(): Process 3 initialized
INFO: results_utils.create_iteration_results_file(): Created iteration results file: /home/darri/data/CONFLUENCE_data/domain_Bow_at_Banff/optimisation/run_2_parallel_iteration_results.csv
INFO: parallel_utils.run(): Worker 2 started and waiting for tasks
INFO: results_utils.create_iteration_results_file(): Created iteration results file: /home/darri/data/CONFLUENCE_data/domain_Bow_at_Banff/optimisation/run_2_parallel_iteration_results.csv
INFO: parallel_utils.run(): Worker 3 started and waiting for tasks
INFO: parallel_parameter_estimation.main(): Starting optimization with NSGA-II algorithm
INFO: parallel_utils.run(): Worker 1 started and waiting for tasks
INFO: parallel_parameter_estimation.main(): Optimization metric: KGE
I